In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_json("./prompts/humaneval-ts-reworded.jsonl", lines=True)
df.head()

,name,language,prompt,doctests,original,prompt_terminology,tests,stop_tokens
0,HumanEval_0_has_close_elements,ts,"//Check if in given array of numbers, are any ...",transform,/home/arjun/repos/nuprl/MultiPL-E/datasets/../...,reworded,declare var require: any;\nconst assert = requ...,"[\nfunction , \n/*, \n//, \nclass]"
1,HumanEval_100_make_a_pile,ts,"//Given a positive integer n, you have to make...",transform,/home/arjun/repos/nuprl/MultiPL-E/datasets/../...,reworded,declare var require: any;\nconst assert = requ...,"[\nfunction , \n/*, \n//, \nclass]"
2,HumanEval_101_words_string,ts,//You will be given a string of words separate...,transform,/home/arjun/repos/nuprl/MultiPL-E/datasets/../...,reworded,declare var require: any;\nconst assert = requ...,"[\nfunction , \n/*, \n//, \nclass]"
3,HumanEval_102_choose_num,ts,//This function takes two positive numbers x a...,transform,/home/arjun/repos/nuprl/MultiPL-E/datasets/../...,reworded,declare var require: any;\nconst assert = requ...,"[\nfunction , \n/*, \n//, \nclass]"
4,HumanEval_103_rounded_avg,ts,"//You are given two positive integers n and m,...",transform,/home/arjun/repos/nuprl/MultiPL-E/datasets/../...,reworded,declare var require: any;\nconst assert = requ...,"[\nfunction , \n/*, \n//, \nclass]"


In [3]:
# The first variable is between "(" and ":"
first_var_re = re.compile(r"\(([^:]+):") 
# The remaining variables are btween "," and ":"
remaining_vars_re = re.compile(r",([^:]+):")

def update_prompt(p):
    lines = p.strip().split("\n")
    results = lines[1:-1]
    last_line = lines[-1]
    # Find all variables and get their positiosn
    all_vars = [first_var_re.search(last_line)]
    all_vars.extend(re.finditer(remaining_vars_re, last_line))
    all_vars = list(reversed(all_vars))
    var_ix = len(all_vars) - 1
    for item in all_vars:
        (start, end) = item.span()
        start = start + 1
        end = end - 1
        # Rename the variable at item.span with __tmp{var_ix}
        last_line = last_line[:start] + f"__tmp{var_ix}" + last_line[end:]
        var_ix -= 1
    results.append(last_line)
    return "\n".join(results)

In [4]:
print("Before:")
print(df.iloc[0]["prompt"])
updated_df = df.copy()
updated_df["prompt"] = updated_df["prompt"].apply(update_prompt)
print("After:")
print(updated_df.iloc[0]["prompt"])

Before:
//Check if in given array of numbers, are any two numbers closer to each other than
// given threshold.
// >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
// false
// >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
// true
function has_close_elements(numbers: number[], threshold: number): boolean {

After:
// given threshold.
// >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
// false
// >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
// true
function has_close_elements(__tmp0: number[],__tmp1: number): boolean {


In [5]:
updated_df.to_json("multipl_ts_renamed.jsonl", lines=True, orient="records")